In [1]:
class ValidString:
    def __set_name__(self, owner, name):
        print(f"__set_name__ called from owner={owner} with property_name = {name}")

In [2]:
ValidString()

In [3]:
class Person:
    name = ValidString()

__set_name__ called from owner=<class '__main__.Person'> with property_name = name


As you can see __set_name__ was called when the Person class was created. This is the only time it gets called.

The main advantage of this is that we can capture the property name:



In [4]:
class ValidString:
    def __set_name__(self, owner, name):
        print(f"__set_name__ called from owner={owner} with property_name = {name}")
        self.property_name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            print(f"__get__ called for property {self.property_name} of instance {instance}")

In [5]:
class Person:
    first_name = ValidString()
    last_name = ValidString()

__set_name__ called from owner=<class '__main__.Person'> with property_name = first_name
__set_name__ called from owner=<class '__main__.Person'> with property_name = last_name


In [6]:
p = Person()
p.first_name

__get__ called for property first_name of instance <__main__.Person object at 0x000001E260C37D10>


In [7]:
p.last_name

__get__ called for property last_name of instance <__main__.Person object at 0x000001E260C37D10>


So basically we know which property name was assigned to the instance of the descriptor.

## Better Error Message

In [8]:
class ValidString:
    def __init__(self, min_length=None):
        self.min_length = min_length
    def __set_name__(self, owner, name):
        self.property_name = name
    def __set__(self, instance, value):
        if not  isinstance(value,str):
            raise ValueError(f"{self.property_name} must be string")
        elif self.min_length is not None and len(value) < self.min_length:
            raise ValueError(f"{self.property_name} must have at least {self.min_length} characters.")
        key = "_" + self.property_name
        setattr(instance,key,value)
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            key = "_" + self.property_name
            return getattr(instance,key,None)

In [9]:
class Person:
    first_name = ValidString(1)
    last_name = ValidString(2)

In [10]:
p = Person()

In [11]:
try:
    p.first_name = "some"
    p.last_name = "A"
except ValueError as ex:
    print(ex)

last_name must have at least 2 characters.


In [12]:
try:
    p.first_name = ""
    p.last_name = "A"
except ValueError as ex:
    print(ex)

first_name must have at least 1 characters.


In [13]:
p = Person()
p.first_name = "Alex"

In [14]:
p.__dict__

{'_first_name': 'Alex'}

We also used the property name as the basis for an attribute in the instance itself:

So although this now fixes the issue we saw at the beginning of this section (having the user specify the property name twice), we still have the issue of potentially overwriting an existing instance attribute:



In [15]:
p = Person()
p._first_name = "some date need to be stored"

In [16]:
p.__dict__

{'_first_name': 'some date need to be stored'}

In [17]:
p.first_name = "Alex"

In [18]:
p.__dict__

{'_first_name': 'Alex'}

So that wiped away our data - this is not good, so we need to do something about it.

How about storing the value in the instance using the exact same name?

In [19]:
class BankAccount:
    apr = 10


In [20]:
b = BankAccount()

In [21]:
b.apr

10

In [22]:
b.__dict__

{}

In [23]:
b.apr = 100

In [24]:
b.apr , b.__dict__

(100, {'apr': 100})

So as you can see, the descriptor is a class attribute. So if we store the value under the same name in the instance, are we not going to run into this shadowing issue where the attribute will now use the attribute in the instance rather than using the class descriptor attribute?

And the answer is it depends!

Data vs non-data descriptors - that distinction is important

## Same name

In [33]:
class ValidString:
    def __init__(self, min_length=None):
        self.min_length = min_length
    def __set_name__(self, owner, name):
        self.property_name = name
    def __set__(self, instance, value):
        if not  isinstance(value,str):
            raise ValueError(f"{self.property_name} must be string")
        elif self.min_length is not None and len(value) < self.min_length:
            raise ValueError(f"{self.property_name} must have at least {self.min_length} characters.")
        instance.__dict__[self.property_name] = value
        #setattr(instance,self.property_name,value) # this go infinite loop
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            print(f" descriptor __get__ called")
            return instance.__dict__.get(self.property_name,None)
            # return getattr(instance,self.property_name,None) # infinite loop

In [34]:
class Person:
    first_name = ValidString(1)
    last_name = ValidString(2)

In [35]:
p =Person()

In [36]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              'first_name': <__main__.ValidString at 0x1e26112db90>,
              'last_name': <__main__.ValidString at 0x1e25f6e8e10>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [37]:
p.__dict__

{}

In [38]:
p.first_name = "Alex"

In [39]:
p.__dict__

{'first_name': 'Alex'}

In [40]:
p.first_name

 descriptor __get__ called


'Alex'